# Scraping data of all Provider IDs from webpricer.cms.gov 

Attempting to scrape information from a webpage whose output changes dynamically as we enter text in a field

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

Atttempting to get rid of initial popup:

In [11]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')

driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


Experimenting to find the right web element and regex used to extract the data:

In [13]:
ID = "100030"

provider_list = [ID]

In [14]:
driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)

In [ ]:
table1 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody")[0].text
table2 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody")[0].text
table = table1 + "\n" + table2
print(table)

In [10]:
# print(table.split("\n"))
print(len(table.split("\n")))
column_names = ["ProviderID"]
for i in table.split("\n"):
    column_names.append(i[:i.rfind(' ')])
# print(column_names)
for i in range(len(column_names)):
    column_names[i] = column_names[i].replace(' ','_')
print(column_names)

60
['ProviderID', 'Fiscal_year_begin_date', 'Report_date', 'Termination_Date', 'Waiver_code', 'Intermediary_number', 'Provider_type', 'Current_Census_division', 'Change_code_for_reclass', 'Actual_Geographic_MSA', 'Wage_Index_MSA', 'Standardized_amount_MSA', 'Sole_community_or_Medicare', 'LUGAR', 'Temporary_relief_indicator', 'Federal_PPS_Blend_indicator', 'Facility_specific_rate', 'Cost_of_living_adjustment', 'Intern_to_bed_ratio', 'Bed_size', 'Cost-to-charge_ratio', 'Case_mix_index', 'SSI_ratio', 'Medicaid_ratio', 'Provider_PPS_period', 'Special_provider_update_factor', 'Operating_DSH_percent', 'Fiscal_year_end_date', 'Special_wage_index', 'Special_payment_indicator', 'Hospital_quality_indicator', 'Geographic_location_CBSA', 'Reclassification_CBSA', 'Standardized_amount_CBSA', 'Total_pass_through_amount_and_miscellaneous', 'Pass_through_amount_capital', 'Pass_through_amount_direct', 'Pass_through_amount_organ', 'Capital_PPS_pay_code', 'Capital_hospital_specific_rate', 'Capital_old_har

In [37]:
provider_list = [ID]
for i in table.split("\n"):
    provider_list.append(i[i.rfind(' ')+1:])
print(len(provider_list))

61


Importing the inpatient provider numbers and checking to see if it loaded correctly:

In [5]:
inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")
inpatient.head(5)

,Inpatient,CCN
0,NaN,10001
1,NaN,10005
2,NaN,10006
3,NaN,10007
4,NaN,10008


Alternatively, if using all the Provider IDs:

*Obtained by querying* `SELECT [Federal_Provider_Number] FROM [ProviderDataRegistry].[dbo].[CMS_ProviderInfo]`

In [22]:
all_providers = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")
all_providers.head(5)

,Federal_Provider_Number
0,015009
1,015010
2,015012
3,015014
4,015015


Run this block to perform web scraping:

In [38]:
# one hour for ~3000 provider ID_s

# Changing the data to all the providers
# inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")
all_providers = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")

try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

ID_list = all_providers["Federal_Provider_Number"].to_list()
ID_list = [str(x) for x in ID_list]
for i in range(len(all_providers)):
    while(len(ID_list[i]) < 6):  
        ID_list[i] = "0" + ID_list[i]
     
final_list = []
df = pd.DataFrame([])
count = 1
l = len(ID_list)
bad_ID = []

for ID in ID_list:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)

    # Since this data has provider IDs that dont return a match, the program logs and skips an ID if it does not get a response within 10 seconds
    stop = False
    while True:
        try:
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    stop = False
    while True:
        try:
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print("\ (•◡•) /   Extracted data from Provider ID: " + ID + "!   \ (•◡•) /", end = "  ")
    print("  <<"+str(count)+" out of "+str(l)+" provider's info taken!>>", end = "\r")
    count += 1
    

df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0120D953+2414931]
	Ordinal0 [0x0119F5E1+1963489]
	Ordinal0 [0x0108C6B8+837304]
	Ordinal0 [0x01074120+737568]
	Ordinal0 [0x010D6E29+1142313]
	Ordinal0 [0x010E3FB2+1195954]
	Ordinal0 [0x010D42B6+1131190]
	Ordinal0 [0x010AE860+976992]
	Ordinal0 [0x010AF756+980822]
	GetHandleVerifier [0x0147CC62+2510274]
	GetHandleVerifier [0x0146F760+2455744]
	GetHandleVerifier [0x0129EABA+551962]
	GetHandleVerifier [0x0129D916+547446]
	Ordinal0 [0x011A5F3B+1990459]
	Ordinal0 [0x011AA898+2009240]
	Ordinal0 [0x011AA985+2009477]
	Ordinal0 [0x011B3AD1+2046673]
	BaseThreadInitThunk [0x76986739+25]
	RtlGetFullPathName_UEx [0x77B38FEF+1215]
	RtlGetFullPathName_UEx [0x77B38FBD+1165]


The above takes about 5 hours to run and chrome crashed after ID 365271, this is most probably just an error in the browser or my computer. Saving the data so far, then re-starting the loop from where we left off:

Saving data from incomplete run just in case:

In [43]:
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,015009,07/01/2021,03/31/2022,--,N,05901,38,5,N,01,...,0,0,--,$0.00,--,0,--,01,0,--
1,015010,07/01/2021,03/31/2022,--,N,10111,38,5,--,01,...,0,0,--,$0.00,--,0,--,01,0,--
2,015012,10/01/2021,03/31/2022,--,N,10111,38,5,--,01,...,0,0,--,$0.00,--,0,--,01,0,--
3,015014,07/01/2021,03/31/2022,--,N,05901,38,5,N,1000,...,0,0,--,$0.00,--,0,--,01,0,--
4,015015,07/01/2021,03/31/2022,--,N,05901,38,5,N,1000,...,0,0,--,$0.00,--,0,--,01,0,--
5,015016,07/01/2021,03/31/2022,--,N,10111,38,5,--,3440,...,0,0,N,$0.00,--,0,--,01,0,--
6,015019,01/01/2021,03/31/2022,--,N,04011,38,6,--,5240,...,0,0,--,$0.00,--,0,01051,01,0,--
7,015023,07/01/2021,03/31/2022,--,N,10111,38,5,--,01,...,0,0,--,$0.00,--,0,--,01,0,--
8,015024,07/01/2021,03/31/2022,--,N,10111,38,5,--,3440,...,0,0,N,$0.00,--,0,--,01,0,--
9,015027,07/01/2021,03/31/2022,--,N,05901,38,5,N,5240,...,0,0,--,$0.00,--,0,--,01,0,--


In [44]:
df.to_csv("All_provider_9681.csv")

In [17]:
for i in range(len(final_list)):
    if len(final_list[i]) != 61:
        final_list[i] = final_list[i][:61] 
        print(final_list[i])



['050137', '01/01/2022', '03/31/2022', '--', 'N', '01911', '07', '9', '--', '0000', '0000', '0000', '--', '--', '--', '--', '$2,788.67', '1', '0.0114', '183', '0.338', '1.3803', '0.0871', '0.1659', '--', '--', '--', '12/31/2016', '--', 'Y', '1', '31084', '37100', '05', '$3.78', '--', '$3.78', '--', 'C', '384.57', '0', '0', '0.008', '--', '0.0233', '0', '--', '--', 'N', '0', '1', '1', '0', 'N', '$4,029.72', '--', '0', '06037', '05', '0', '--']


In [42]:
print(bad_ID)
df_error = pd.DataFrame(bad_ID)
df_error.to_csv("Bad_IDs_9681.csv")

['01A193', '01A208', '04A158', '04A293', '04E090', '05A021', '05A024', '05A027', '05A109', '05A110', '05A134', '05A137', '05A147', '05A164', '05A183', '05A192', '05A208', '05A263', '05A264', '05A269', '05A277', '05A290', '05A292', '05A315', '05A331', '05A340', '05A355', '05A357', '05A360', '05A364', '05A371', '05A396', '05A397', '05A408', '05A427', '06A088', '06A171', '06A172', '06A173', '06A175', '06A185', '06A190', '06A192', '06A196', '08A006', '08A011', '08A015', '08A020', '09E020', '106110', '11A186', '11A200', '14A057', '14A357', '14A383', '14A453', '14A539', '14E095', '14E169', '14E177', '14E212', '14E247', '14E264', '14E306', '14E322', '14E345', '14E361', '14E392', '14E579', '14E701', '14E812', '14E847', '14E848', '14E888', '15A011', '15E064', '15E247', '15E667', '15E681', '15E683', '165788', '16A001', '16A002', '16E016', '16E050', '16E071', '16E170', '16E176', '16E263', '16E277', '16E637', '16E638', '16E668', '16E718', '16E728', '16E756', '16F001', '16F002', '17A020', '17A029',

Continuing the loop:

In [ ]:
# one hour for ~3000 provider ID_s

# Changing the data to all the providers
# inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")

#all_providers = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")

try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

# ID_list = all_providers["Federal_Provider_Number"].to_list()
# ID_list = [str(x) for x in ID_list]
# for i in range(len(all_providers)):
#     while(len(ID_list[i]) < 6):  
#         ID_list[i] = "0" + ID_list[i]
     
# final_list = []
# df = pd.DataFrame([])
count = 9681
# l = len(ID_list)
# bad_ID = []

for ID in ID_list:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)

    stop = False
    while True:
        try:
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    stop = False
    while True:
        try:
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print("\ (•◡•) /   Extracted data from Provider ID: " + ID + "!   \ (•◡•) /", end = "  ")
    print("  <<"+str(count)+" out of "+str(l)+" provider's info taken!>>", end = "\r")
    count = count + 1

df2 = pd.DataFrame(final_list, columns= column_names)
df2.head(10)

One of the entries has 91 columns instead of 61, I think this happened because I accidentally resized the window during the runtime and it changed the html while it was being read.
Luckily, the first 61 columns are exactly what they should be and we can simply ditch the extra columns and proceed:

In [52]:
for i in range(len(final_list)):
    if len(final_list[i]) != 61:
        print(i)

# output was 13402

print(final_list[13402])
print(len(final_list[13402]))


13402
['535022', '07/01/2021', '03/31/2022', '--', 'N', '03001', '38', '0', '--', '53', '--', '--', '--', '--', '--', '4', '$207.27', '1', '--', '160', '0', '--', '0', '0', '--', '--', '--', '06/30/2010', '--', '--', '1', '53', '--', '--', '$0.00', '--', '--', '--', '--', '0', '0', '0', '0', '--', '--', '0', '--', '--', 'N', '0.992', '0', '0', '0', '--', '$0.00', '--', '0', '--', '53', '0', '--', '53', '--', '--', '$0.00', '--', '--', '--', '--', '0', '0', '0', '0', '--', '--', '0', '--', '--', 'N', '0.992', '0', '0', '0', '--', '$0.00', '--', '0', '--', '53', '0', '--']
91


In [53]:
final_list[13402] = final_list[13402][1:61]

In [54]:

df2 = pd.DataFrame(final_list, columns= column_names)
print(df2.shape)

(14918, 61)


Check if all entries are accounted for:

In [60]:
if df2.shape[0] + len(bad_ID) == len(all_providers["Federal_Provider_Number"].to_list()):
    print("All Provider IDs accounted for!")
else:
    print("There exist "+str(len(all_providers["Federal_Provider_Number"].to_list()) - (df2.shape[0] + len(bad_ID))) +" unaccounted IDs")
    print(df2.shape[0], end = " + ")
    print(len(bad_ID), end = " = "+str(df2.shape[0] + len(bad_ID))+", should be "+str(len(all_providers["Federal_Provider_Number"].to_list()))+"\n")
    print()

There exist -1 unaccounted IDs
14918 + 307 = 15225, should be 15224



We have an extra entry, probably repeated the row that we stopped on then restarted from. Checking to see where the duplicate was:

In [62]:
uniq = []
for i in final_list:
    if i[0] not in uniq:
        uniq.append(i[0])
    else:
        print(i)

uniq2 = []
for i in bad_ID:
    if i in uniq:
        print(i + " in final and bad")
    if i in uniq2: 
        print(i + " occurs twice in bad")
    if i not in uniq2:
        uniq2.append(i)


365272 in final and bad


The problematic entry (ID:365272) was the one which the loop stopped on when Chrome crashed. We can see the extra entry was created by this accidentally being added to the bad IDs.

Removing the bad entry and saving the final data:

In [ ]:
# Put in a separate block because it can only be run once
bad_ID.remove("365272")

Change the -- entries used to represent the missing values to empty strings: 

In [80]:
for i in range(len(final_list)):
    for j in range(len(final_list[i])):
        if final_list[i][j] == "--":
            final_list[i][j] = ""

In [81]:
pd.DataFrame(final_list, columns = column_names).to_csv("CMS_provider_directory_all.csv")
pd.DataFrame(bad_ID, columns = ["Bad_IDs"]).to_csv("CMS_provider_directory_all_bad_IDs.csv")

### Ignore the following blocks: 

On observing the CSV, we can see that bad_ID used numerical values rather than strings and ended up using scientific notation. We can fix this as we can get the bad values by seeing which values from the original CSV are not present in final_list:  

In [ ]:
new_bad_ID = []
full = all_providers["Federal_Provider_Number"].to_list()
full = [str(x) for x in full]
print(full)

final_IDs = []
for i in final_list:
    final_IDs.append(i[0])

for i in full:
    if i not in final_IDs:
        new_bad_ID.append(i)

pd.DataFrame(new_bad_ID, columns = ["Bad_IDs"]).to_csv("CMS_provider_directory_all_bad_IDs.csv")